In [2]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('agg')
import matplotlib.pyplot as plt
%matplotlib inline
import os
from tqdm import tqdm
import talib

In [3]:
# 这里导入数据
df = pd.read_csv("../数据获取/聚宽数据/最新数据.csv")
df.columns = ['id','i', 'Date','Open','High','Low','Close','Volume']
df['id2'] =df['id'].shift()
df['i'] = range(len(df))
ids = list(df.loc[df['id'] != df['id2'], 'i']) # 这个是单个股票序号的起始。
ids.append(len(df)-1) # 添加最后1天。
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../数据获取/聚宽数据/最新数据.csv'

In [ ]:
after_days = 20 # 未来5天的
after_up = [[] for _ in range(after_days)]  # 几天关于上涨的
with tqdm(total=len(set(df['id']))) as pbar:
    for j in range(len(ids)-1):
        # 对每一个股票进行遍历
        df2 = df.iloc[ids[j]:ids[j+1],:].copy() # 这里筛选一只股票。
        for i in range(after_days):
            df2['Close{}'.format(i+1)] = df2['Close'].shift(0-(i+1)) # 未来的收盘价移动
        df2.dropna(axis=0, how='any', inplace=True) # 我这里不需要这个空值，直接删除吧。
        for i in range(after_days):
            after_up[i].extend(((df2['Close{}'.format(i+1)]-df2['Close'])/df2['Close']*100).tolist())
        pbar.update(1)


In [ ]:
for i in range(after_days):
    print("往后{:2d}, \t平均涨幅:{:.2f}%, \t胜率:{:.2f}%, \t涨9%以上的概率:{:.2f}%,\t涨0%-9%的概率:{:.2f}%".format(
        i+1,
        sum(after_up[i])/len(after_up[i]),
        len([i for i in after_up[i] if i > 0])/len(after_up[i])*100,
        len([i for i in after_up[i] if i > 9])/len(after_up[i])*100,
        len([i for i in after_up[i] if i > 0 and i < 9])/len(after_up[i])*100,
        len([i for i in after_up[i] if i > 0])/len(after_up[i])*100
    ))

# 小结

这个总结就是，对于平均一致股票而言，1-20天，上涨的概率差不多是近50%，但是随着时间的推移，比如到20天，涨9%的概率是超过20%了。